In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

# Ensure your Google Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks/GhostNet')  # Add the path to your ghostnetN1.py file
from ghostnetN1 import GhostNet_N  # Now Python should find the file

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image, UnidentifiedImageError
import os

class SafeImageFolder(datasets.ImageFolder):
    def __init__(self, root, transform=None):
        super().__init__(root=root, transform=transform)

    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            # Try opening the image file
            sample = Image.open(path)
            sample = sample.convert("RGB")  # Ensure image is in RGB format
        except (UnidentifiedImageError, OSError) as e:
            print(f"Skipping corrupted image: {path}")
            #Print out the index of the image that I am skipping. Ignoring the image will keep all other image indexes the same.
            raise IndexError(f"Corrupted image at index {index}, skipping.")

        if self.transform is not None:
            sample = self.transform(sample)

        return sample, target


# Define training configuration and model
cfgs = [
    [[3, 16, 16, 0, 1, 0]],  # No CBAM, spatial_kernel set to 0
    [[3, 48, 24, 0, 2, 0]],  # No CBAM, spatial_kernel set to 0
    [[3, 72, 24, 0, 1, 0]],  # No CBAM, spatial_kernel set to 0
    [[5, 72, 40, 0.25, 2, 5]],  # CBAM active, spatial_kernel = 5
    [[5, 120, 40, 0.25, 1, 7]],  # CBAM active, spatial_kernel = 7
    [[3, 240, 80, 0, 2, 0]],  # No CBAM, spatial_kernel set to 0
    [[3, 200, 80, 0, 1, 0],  # No CBAM, spatial_kernel set to 0
     [3, 184, 80, 0, 1, 0],  # No CBAM, spatial_kernel set to 0
     [3, 184, 80, 0, 1, 0],  # No CBAM, spatial_kernel set to 0
     [3, 480, 112, 0.25, 1, 5],  # CBAM active, spatial_kernel = 5
     [3, 672, 112, 0.25, 1, 5]],  # CBAM active, spatial_kernel = 5
    [[5, 672, 160, 0.25, 2, 3]],  # CBAM active, spatial_kernel = 3
    [[5, 960, 160, 0, 1, 0],  # No CBAM, spatial_kernel set to 0
     [5, 960, 160, 0.25, 1, 3],  # CBAM active, spatial_kernel = 3
     [5, 960, 160, 0, 1, 0],  # No CBAM, spatial_kernel set to 0
     [5, 960, 160, 0.25, 1, 3]]  # CBAM active, spatial_kernel = 3
]

# Model initialization
model = GhostNet_N(cfgs, num_classes=1000, width=1.0, dropout=0.2)

# Dataset and DataLoader setup (load data from Google Drive)
train_dir = '/content/drive/MyDrive/ColabNotebooks/GhostNet/ImageNet/dir/train'
val_dir = '/content/drive/MyDrive/ColabNotebooks/GhostNet/ImageNet/dir/val'

# Define normalization parameters (use ImageNet's mean and std)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Apply transformations for training and validation
train_transform = transforms.Compose([
    transforms.Resize(256),  # Resize images to 256
    transforms.CenterCrop(224),  # Crop to 224x224 for ImageNet models
    transforms.ToTensor(),
    normalize,
])

val_transform = transforms.Compose([
    transforms.Resize(256),  # Resize images to 256
    transforms.CenterCrop(224),  # Crop to 224x224 for ImageNet models
    transforms.ToTensor(),
    normalize,
])

# Load training and validation datasets using ImageFolder
train_dataset = SafeImageFolder(root=train_dir, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)

val_dataset = SafeImageFolder(root=val_dir, transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()  # Zero gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Validation step
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        print(f"Validation Accuracy: {100 * correct / total:.2f}%")

# Save the trained model to Google Drive
torch.save(model.state_dict(), '/content/drive/MyDrive/ColabNotebooks/GhostNet/ghostnet_cbamN1.pth')
print("Model saved to Google Drive as 'ghostnet_cbamN1.pth'")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch [1/10], Loss: 6.7944
Validation Accuracy: 0.10%
Epoch [2/10], Loss: 6.6102
Validation Accuracy: 0.28%
Epoch [3/10], Loss: 6.4710
Validation Accuracy: 0.64%
Epoch [4/10], Loss: 6.2312
Validation Accuracy: 1.07%
Epoch [5/10], Loss: 6.0055
Validation Accuracy: 1.73%
Epoch [6/10], Loss: 5.8560
Validation Accuracy: 2.27%
Epoch [7/10], Loss: 5.7275
Validation Accuracy: 2.29%
Epoch [8/10], Loss: 5.5902
Validation Accuracy: 2.70%


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    train_dir,
    transform=transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

# Try loading a batch
try:
    images, labels = next(iter(train_loader))
    print(f"✅ Successfully loaded a batch: {images.shape}")
except Exception as e:
    print(f"❌ Error loading training dataset: {e}")


✅ Successfully loaded a batch: torch.Size([32, 3, 224, 224])
